# Fine tuning with unlabeld dataset 

### Training Rasnet18 with self supervised SimCLR technique

In [1]:
!pip3 install lightning torchvision matplotlib pandas

In [2]:
# from torchvision.model import resnet18
import lightning
import matplotlib.pyplot as plt 
import pandas as pd
import torchvision
import numpy as np
from shared import * 
from lightning.pytorch.loggers import CSVLogger


In [3]:
torch.manual_seed(123)

In [4]:
pytorch_model = torch.hub.load('pytorch/vision', 'resnet18', weights=False)
pytorch_model

Using cache found in /Users/parthmishra/.cache/torch/hub/pytorch_vision_main
/opt/anaconda3/lib/python3.13/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
pytorch_model.fc = torch.nn.Sequential(
    torch.nn.Linear(512, 512),
    torch.nn.ReLU(),
    torch.nn.Linear(512, 10)
)

In [ ]:
pytorch_model.load_state_dict(torch.load('resnet18_cifar10.pth'))


In [ ]:

# Data Loading
dm = Cifar10DataModule(
    height_width=(128,128),
    batch_size=256,
    num_workers=5
    )

LightningModel_duplicate = LightningModel(
    model=pytorch_model,
    learning_rate=0.0005
)

trainer = L.Trainer(
    max_epochs=50,
    accelerator="auto",
    devices="auto",
logger=CSVLogger(save_dir="logs/", name="mine_model"),deterministic=True
)

trainer.fit(LightningModel_duplicate, dm)